In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
warnings.simplefilter('ignore', np.RankWarning)

import logging, sys
logging.disable(sys.maxsize)

# Load Data
### Загрузка данных

In [4]:
try: 
    from tsad.base.datasets import load_skab
except:
    import sys
    sys.path.append('../')
    from tsad.base.datasets import load_skab

In [5]:
dataset = load_skab()
df = dataset.frame
df = df.reset_index(level=[0])
df = df[df['experiment']=='valve1/6']
df = df.drop(columns='experiment')
df.shape

(1154, 10)

In [6]:
#TODO use task in pipeline to resample dataframe
df = df.resample('1s').mean().ffill()
df.shape

(1200, 10)

In [7]:
features = dataset.feature_names
target = dataset.target_names[0]

# Feature Generation
### Генерация признаков

## Create a FeatureGenerationTask Instance
#### Создание экземпляра задачи FeatureGenerationTask

The `FeatureGenerationTask` is designed to facilitate feature generation from a given DataFrame based on a specified configuration. It is responsible for generating features from time series data according to a user-defined or default configuration.

__RU:__ _Функция FeatureGenerationTask предназначена для облегчения генерации признаков из заданного фрейма данных на основе заданной конфигурации. Она отвечает за генерацию признаков из данных временных рядов в соответствии с пользовательской конфигурацией или конфигурацией по умолчанию._

In [8]:
try: 
    from tsad.tasks.feature_generation import FeatureGenerationTask
except:
    import sys
    sys.path.append('../')
    from tsad.tasks.feature_generation import FeatureGenerationTask


In [9]:
feature_generation_task = FeatureGenerationTask(config=None, features=features)

- `config` (optional): Configuration for feature generation, provided as a list of dictionaries. If not provided, default configurations will be used.
- `features` (optional): A list of features to consider. If not specified, all available columns in the DataFrame will be used.

__RU:__
- `config` (необязательно): Конфигурация для генерации признаков, предоставляемая в виде списка словарей. Если не указано, будут использоваться конфигурации по умолчанию.
- `features` (необязательно): Список признков, которые следует учитывать. Если не указано, будут использоваться все доступные столбцы во фрейме данных.

## Default configuration
#### Конфигурация по умполчанию

__Default feature generation functions__:

By default, this method uses the [EfficientFCParameters](https://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.settings.EfficientFCParameters) function for feature generation, which provides most common optimized set of feature extraction functions.


__Default windows__:

The default window sizes for feature generation are determined based on the index frequency of the input DataFrame (_freq_df_). The following window sizes are used:

- Window 1: 4 times the frequency of the DataFrame (4 * _freq_df_)
- Window 2: 10 times the frequency of the DataFrame (10 * _freq_df_)

These window sizes are selected to capture a range of temporal patterns in the time series data.

__Функции генерации признаков по умолчанию__:

По умолчанию этот метод использует функцию [EfficientFCParameters](https://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.settings.EfficientFCParameters) для генерации признков, которая предоставляет наиболее распространенный оптимизированный набор функций извлечения признаков.


__Окна по умолчанию__:

Размеры окон по умолчанию для генерации признков определяются на основе индексной частоты фрейма входных данных (_freq_df_). Используются следующие размеры окон:

- Окно 1: частота фрейма данных в 4 раза больше (4 * _freq_df_)
- Окно 2: частота фрейма данных в 10 раз больше (10 * _freq_df_)

Эти размеры окон выбраны для того, чтобы охватить диапазон временных закономерностей в данных временных рядов.

### Fitting the Task
#### Обучение

Now that you have initialized the task, it's time to fit it to your input DataFrame. The `fit` method will perform feature generation based on your configuration. Here's how to use it:

__RU:__ _Теперь, когда вы инициализировали задачу, пришло время "подогнать" ее к входному фрейму данных. Метод fit выполнит генерацию признаков на основе вашей конфигурации. Вот как его использовать:_

In [10]:
%%time
# Fit the FeatureGenerationTask to your DataFrame
df_generated, generation_result = feature_generation_task.fit_predict(df)
df_generated.shape

CPU times: user 1.79 s, sys: 482 ms, total: 2.28 s
Wall time: 17.2 s


(1200, 4342)

- `df` should be your input DataFrame containing the data you want to generate features from.
- `df_generated` will be a new DataFrame containing the original columns plus the generated features.
- `generation_result` will hold information about the generated features.

- `df` должен быть вашим входным фреймом данных, содержащим данные, из которых вы хотите сгенерировать признаки.
- `df_generated` будет новым фреймом данных, содержащим исходные столбцы плюс сгенерированные признаки.
- `generation_result` будет содержать информацию о сгенерированных признаках.

### Making Predictions
#### Формирование прогнозов

If you want to use these generated features for predictions, you can do so easily using the `predict` method:

__RU:__ _Если вы хотите использовать эти сгенерированные признаки для прогнозирования, вы можете легко сделать это, используя метод "predict".:_

In [11]:
%%time
df_predicted, _ = feature_generation_task.predict(df, generation_result)
df_predicted.shape

CPU times: user 1.78 s, sys: 485 ms, total: 2.27 s
Wall time: 16.4 s


(1200, 4342)

## Custom configuration
#### Пользовательская конфигурация

When performing feature generation, you have the flexibility to define a custom configuration tailored to your specific needs. This custom configuration allows you to select a set of feature extraction functions, specify the series (columns) to which these functions will be applied, and define the windows for calculating these features.

__RU:__ _При выполнении генерации признаков у вас есть возможность гибко определить пользовательскую конфигурацию, адаптированную к вашим конкретным потребностям. Эта пользовательская конфигурация позволяет вам выбрать набор функций извлечения признаков, указать ряды (столбцы), к которым будут применяться эти функции, и определить окна для вычисления._

### Custom Configuration Example
#### Пример пользовательской конфигурации

If you need to customize the feature generation process, you can provide your own configuration. The config parameter allows you to define a list of dictionaries, each specifying a set of features to generate.

__RU:__ _Если вам нужно настроить процесс генерации признков, вы можете предоставить свою собственную конфигурацию. Параметр config позволяет вам определить список словарей, в каждом из которых указан набор признаков для генерации._

In [12]:
import scipy.stats as ss

from tsflex.features import FuncWrapper
from tsflex.features.utils import make_robust

In [13]:
def slope(x): return (x[-1] - x[0]) / x[0] if x[0] else 0
def abs_diff_mean(x): return np.mean(np.abs(x[1:] - x[:-1])) if len(x) > 1 else 0
def diff_std(x): return np.std(x[1:] - x[:-1]) if len(x) > 1 else 0

funcs = [make_robust(f) for f in [np.min, np.max, np.std, np.mean, slope, ss.skew, abs_diff_mean, diff_std, sum, len,]]

custom_config = [
    {"functions": funcs,
     'series_names': ['Pressure', 'Temperature'],
     "windows": ["10s", "60s"],
    }
]

- `functions`: This is a list of feature extraction functions that will be applied to the selected series. These functions are defined in the `funcs` list, which includes functions like minimum, maximum, standard deviation, mean, slope, skewness, and more. You can customize this list to include the specific functions that are relevant to your analysis.

- `series_names`: This is a list of column names in your DataFrame to which the feature extraction functions will be applied. In this example, the functions will be applied to the `Pressure` and `Temperature` series. You can modify this list to include the names of the series you want to analyze.

- `windows`: This is a list of window sizes for feature calculation. In this example, two window sizes are specified: "1s" (1 second) and "60s" (60 seconds). These window sizes determine how the time series data will be segmented for feature extraction. Adjust these window sizes based on your analysis requirements.

__RU:__

- `functions`: это список функций извлечения признаков, которые будут применены к выбранной серии. Эти функции определены в списке `func`, который включает в себя такие функции, как минимум, максимум, стандартное отклонение, среднее значение, наклон, асимметрия и другие. Вы можете настроить этот список, включив в него конкретные функции, имеющие отношение к вашему анализу.

- `series_names`: это список имен столбцов в вашем DataFrame, к которым будут применяться функции извлечения признаков. В этом примере функции будут применены к сериям `Pressure`и `Temperature`. Вы можете изменить этот список, включив в него названия серий, которые вы хотите проанализировать.

- `windows`: это список размеров окон для расчета объектов. В этом примере указаны два размера окна: «1s» (1 секунда) и «60s» (60 секунд). Эти размеры окна определяют, как данные временных рядов будут сегментированы для извлечения признаков. Настройте размеры этих окон в соответствии с вашими требованиями к анализу.

### Feature Extraction Functions

Feature Extraction functions compute various statistical, temporal, spectral, and other characteristics of time series data. In your feature generation task, you can use a variety of feature extraction functions from libraries like tsfresh, tsfel, numpy, scipy, or even custom functions. 


__RU:__

Функции извлечения признаков

Функции извлечения признаков вычисляют различные статистические, временные, спектральные и другие характеристики данных временных рядов. В задаче создания функций вы можете использовать различные функции извлечения функций из библиотек, таких как tsfresh, tsfel, numpy, scipy или даже пользовательские функции.

__Feature Extraction Categories__:

1. _Statistical Features_:
These features capture statistical properties of the time series data.
Common statistical features include mean, median, standard deviation, skewness, kurtosis, variance, and more.
Example: `np.mean`, `tsfresh.feature_extraction.feature_calculators.median`, etc.

2. _Temporal Features_:
Temporal features describe patterns over time within the time series.
Examples include autocorrelation, mean absolute difference, mean difference, distance, absolute energy, and more.
Example: `tsfresh.feature_extraction.features.autocorr`, `tsfel.features.mean_abs_diff`, etc.


3. _Spectral Features_:
Spectral features provide insights into the frequency domain characteristics of the time series.
These features include wavelet entropy, spectral entropy, power spectral density, and more.
Example: `tsfresh.feature_extraction.features.wavelet_entropy`, `tsfel.features.spectral_entropy`, etc.

4. _Custom Functions_:
You can define custom feature extraction functions tailored to your specific analysis requirements.
These functions can capture domain-specific insights or unique patterns in the data.
Example: Custom functions like `slope(x)`, `abs_diff_mean(x)`, and `diff_std(x)` defined in code.

5. _External Libraries_:
You can leverage external libraries like tsfresh and tsfel for a wide range of pre-defined feature extraction functions.
These libraries offer functions for calculating advanced features such as entropy, time-domain, and frequency-domain features.
Example: `tsfresh.feature_extraction.features.entropy`, `tsfel.features.abs_energy`

__Категории извлечения признаков__:

1. _Статистические характеристики_:
Эти признаки отражают статистические свойства данных временных рядов.
Общие статистические характеристики включают среднее значение, медиану, стандартное отклонение, асимметрию, эксцесс, дисперсию и многое другое.
Пример: `np.mean`, `tsfresh.feature_extraction.feature_calculators.median` и т.д.

2. _Временные характеристики_:
Временные характеристики описывают закономерности во времени внутри временного ряда.
Примеры включают автокорреляцию, среднюю абсолютную разницу, среднюю разницу, расстояние, абсолютную энергию и многое другое.
Пример: `tsfresh.feature_extraction.features.autocorr`, `tsfel.features.mean_abs_diff` и т. д.


3. _Спектральные характеристики_:
Спектральные характеристики дают представление о характеристиках временного ряда в частотной области.
Эти функции включают вейвлет-энтропию, спектральную энтропию, спектральную плотность мощности и многое другое.
Пример: `tsfresh.feature_extraction.features.wavelet_entropy`, `tsfel.features.spectral_entropy` и т. д.

4. _Пользовательские функции_:
Вы можете определить пользовательские функции извлечения признаков, адаптированные к вашим конкретным требованиям анализа.
Эти функции могут фиксировать специфичную для предметной области информацию или уникальные закономерности в данных.
Пример: пользовательские функции, такие как `slope(x)`, `abs_diff_mean(x)` и `diff_std(x)`, определенные в коде.

5. _Внешние библиотеки_:
Вы можете использовать внешние библиотеки, такие как tsfresh и tsfel, для широкого спектра предопределенных функций извлечения признаков.
Эти библиотеки предлагают функции для расчета расширенных функций, таких как энтропия, функции во временной и частотной областях.
Пример: `tsfresh.feature_extraction.features.entropy`, `tsfel.features.abs_energy`

In [14]:
from tsfel.feature_extraction.features import (
    # Some temporal features
    autocorr, mean_abs_diff, mean_diff, distance, zero_cross,
    abs_energy, pk_pk_distance, entropy, neighbourhood_peaks,
    # Some statistical features
    interq_range, kurtosis, skewness, calc_max, calc_median, 
    median_abs_deviation, rms, 
    # Some spectral features
    #  -> Almost all are "advanced" features
    wavelet_entropy
)

tsfel_funcs = [
    # Temporal
    autocorr, mean_abs_diff, mean_diff, distance,
    abs_energy, pk_pk_distance, neighbourhood_peaks,
    # FuncWrapper(entropy, prob="kde", output_names="entropy_kde"),
    # FuncWrapper(entropy, prob="gauss", output_names="entropy_gauss"),
    # # Statistical
    interq_range, kurtosis, skewness, calc_max, calc_median, 
    median_abs_deviation, rms,
    # Spectral
    wavelet_entropy,  
]

# tsfresh
from tsfresh.feature_extraction.feature_calculators import (
    cid_ce,
    variance_larger_than_standard_deviation,
)

tsfresh_funcs=[
        variance_larger_than_standard_deviation,
        FuncWrapper(cid_ce, normalize=True),
    ]

__Choosing Feature Extraction Functions__

When choosing feature extraction functions for your analysis, consider the following factors:

- Relevance: Select functions that are relevant to your analysis goals. For instance, if you're interested in detecting periodicity, consider using autocorrelation or spectral features.

- Computational Efficiency: Consider the computational cost of the functions, especially when dealing with large datasets. Some functions may be computationally expensive.
  
- Domain Knowledge: Leverage your domain knowledge to identify features that have interpretability and meaning in your specific domain.
  
- Customization: Don't hesitate to define custom functions if the standard functions do not capture the patterns you're interested in.

__Выбор функций извлечения признаков__

При выборе функций извлечения признаков для анализа учитывайте следующие факторы:

- Актуальность: выберите функции, соответствующие целям вашего анализа. Например, если вы заинтересованы в обнаружении периодичности, рассмотрите возможность использования автокорреляции или спектральных функций.

- Эффективность вычислений: учитывайте вычислительные затраты функций, особенно при работе с большими наборами данных. Некоторые функции могут быть дорогостоящими в вычислительном отношении.
  
- Знание предметной области: используйте свои знания предметной области для определения функций, которые можно интерпретировать и которые имеют значение в вашей конкретной предметной области.
  
- Настройка: не стесняйтесь определять пользовательские функции, если стандартные функции не отражают интересующие вас шаблоны.

### Applying Custom Configuration
#### Применение пользовательской конфигурации

To apply the custom configuration for feature generation, you can use the FeatureGenerationTask class. Here's an example of how to use it:

__RU:__ _Чтобы применить пользовательскую конфигурацию для создания признаков, вы можете использовать класс FeatureGenerationTask. Вот пример того, как его использовать:_

In [15]:
%%time
# Define your custom configuration
custom_config = [
    {"functions": funcs,
     'series_names': ['Pressure', 'Temperature'],
     "windows": ["10s", "60s"],
    },
    {"functions": tsfel_funcs,
     'series_names': ['Pressure', 'Temperature', 'Thermocouple', 'Voltage'],
     "windows": ["30s", "60s"],
    },
    {"functions": tsfresh_funcs,
     'series_names': ['Pressure', 'Temperature'],
     "windows": ["20s", "60s"],
    },
]

custom_feature_generation_task = FeatureGenerationTask(config=custom_config, features=features)

CPU times: user 7 µs, sys: 2 µs, total: 9 µs
Wall time: 10 µs


In [16]:
%%time
df_generated, generation_result = custom_feature_generation_task.fit_predict(df)
df_generated.shape

CPU times: user 93.4 ms, sys: 54.8 ms, total: 148 ms
Wall time: 1.15 s


(1200, 178)

In [17]:
%%time
df_predicted, _ = custom_feature_generation_task.predict(df, generation_result)
df_predicted.shape

CPU times: user 85.7 ms, sys: 47.7 ms, total: 133 ms
Wall time: 1.1 s


(1200, 178)

# Feature Selection
### Выбор признаков

## Create a FeatureSelectionTask Instance
### Создайте экземпляр FeatureSelectionTask

In [18]:
try: 
    from tsad.tasks.feature_selection import FeatureSelectionTask
except:
    import sys
    sys.path.append('../')
    from tsad.tasks.feature_selection import FeatureSelectionTask



The `FeatureSelectionTask` class is part of `tsad` framework and is used for feature selection. Here's an overview of its main attributes:

- `target`: The target feature name you want to predict.
- `n_features_to_select`: Number of features to select (a fraction or an integer).
- `feature_selection_method`: Method for feature selection. Options include 'univariate', 'tsfresh', 'sequential', or 'frommodel'.
- `feature_selection_estimator`: Estimator used for feature selection (e.g., 'regressor' or 'classifier').
remove_constant_features: Whether to remove constant features.

When creating a `FeatureSelectionTask` instance, you can customize several parameters to tailor the feature selection process to your specific needs. Here's a detailed explanation of these parameters:

- `target` (str):
  - Required: Specify the name of your target feature. This is the feature you want to predict using your machine learning model.
- `n_features_to_select` (float | int | None):
  - Optional: Number of features to select.
  - If you provide an integer value, it will select that exact number of features.
  - If you provide a float value (e.g., 0.2), it will select a fraction of features based on the total number of available features.
  - Setting it to None (default) will not perform any feature selection based on the number of features.
- `feature_selection_method` (str | None):
  - Optional: Method for feature selection.
  - Options include:
    - `univariate`: Perform univariate feature selection based on statistical tests.
    - `tsfresh`: Utilize the `tsfresh` library for automated time series feature selection.
    - `sequential`: Sequential feature selection using an estimator (e.g., RandomForest) for classification or regression.
    - `frommodel`: Select features using an estimator (e.g., RandomForest) for classification or regression.
  - If set to `None` (default), it will use the `frommodel` method by default.
- `feature_selection_estimator` (str | None):
  - Optional: Feature selection estimator.
  - If you choose 'sequential' or 'frommodel' as the feature selection method, you need to specify the estimator.
  - Options depend on your specific use case (e.g., 'classifier' or 'regressor' for classification or regression tasks).
  - If set to None (default), it will use 'regressor' as the default estimator.
- `remove_constant_features` (bool):
  - Optional: Whether to remove constant features from the dataset.
  - Constant features have the same value for all samples and usually don't provide valuable information.
  - Setting it to `True` (default) will remove constant features, and False will keep them in the dataset.

Let's create an instance of `FeatureSelectionTask` and specify the configuration for feature selection.

__RU:__

Класс FeatureSelectionTask является частью платформы tsad и используется для выбора признков. Вот краткий обзор его основных свойств:

- `target`: имя целевого объекта, который вы хотите предсказать.
- `n_features_to_select`: количество признаков для выбора (дробное или целое число).
- `feature_selection_method`: метод выбора признков. Возможные варианты: 'univariate', 'tsfresh', 'sequential', или 'frommodel'.
- `feature_selection_estimator`: оценщик, используемый для выбора признаки (например, 'regressor' или 'classifier').
- `remove_constant_features`: Удалять ли постоянные признаки.

При создании экземпляра `FeatureSelectionTask` вы можете настроить несколько параметров, чтобы адаптировать процесс выбора признаков к вашим конкретным потребностям. Вот подробное объяснение этих параметров:

- `target` (стр):
   - Обязательно: укажите имя целевого признака. Это признак, которуй вы хотите предсказать, используя свою модель машинного обучения.
- `n_features_to_select` (float | int | None):
   - Необязательно: количество признаков для выбора.
   - Если вы укажете целое значение, будет выбрано именно это количество признаков.
   - Если вы укажете плавающее значение (например, 0,2), будет выбрана часть признаков в зависимости от общего количества доступных признаков.
   - Установка значения «Нет» (по умолчанию) не приведет к выбору признаков на основе количества.
- `feature_selection_method` (str | Нет):
   - Необязательно: метод выбора признаков.
   - Опции включают в себя:
     - `univariate`: выполнить одномерный выбор признаков на основе статистических тестов.
     - `tsfresh`: используйте библиотеку `tsfresh` для автоматического выбора признаков временных рядов.
     - `sequential`: последовательный выбор признаков с использованием оценщика (например, RandomForest) для классификации или регрессии.
     - `frommodel`: выберите признаки с помощью оценщика (например, RandomForest) для классификации или регрессии.
   — Если установлено значение «Нет» (по умолчанию), по умолчанию будет использоваться метод «frommodel».
- `feature_selection_estimator` (str | Нет):
   - Необязательно: средство оценки выбора признаков.
   - Если вы выбираете «последовательный» или «из модели» в качестве метода выбора признаков, вам необходимо указать оценщик.
   – Параметры зависят от вашего конкретного варианта использования (например, «классификатор» или «регрессор» для задач классификации или регрессии).
   - Если установлено значение «Нет» (по умолчанию), в качестве оценщика по умолчанию будет использоваться «регрессор».
- `remove_constant_features` (bool):
   - Необязательно: удалять ли постоянные признаки из набора данных.
   - Постоянные признаки имеют одинаковое значение для всех образцов и обычно не дают ценной информации.
   - Установка значения «True» (по умолчанию) приведет к удалению постоянных признаков, а значение False сохранит их в наборе данных.

Давайте создадим экземпляр FeatureSelectionTask и укажем конфигурацию для выбора признаков.

In [19]:
feature_selection_task = FeatureSelectionTask(
    target=target,  # Specify your target feature name
    n_features_to_select=0.2,  # Number of features to select (you can use an integer or fraction)
    feature_selection_method='univariate',  # Choose your feature selection method
    feature_selection_estimator='classifier',  # Choose your estimator (for classification)
    remove_constant_features=True  # Remove constant features
)

## Fit and Select Features
#### Обучить и выбрать признаки

Next, we'll fit the `FeatureSelectionTask` to our dataset and perform feature selection. This step will return a DataFrame with the selected features and a result object for further analysis.

__RU:__ _Далее мы адаптируем `FeatureSelectionTask` к нашему набору данных и выполним выбор признаков. Этот шаг вернет DataFrame с выбранными признаками и объектом результата для дальнейшего анализа._

In [20]:
print(df_generated.shape)
df_selected, result = feature_selection_task.fit_predict(df_generated)
print(df_selected.shape)

(1200, 178)
(1200, 33)


Now that we have our selected features in the `df_selected` DataFrame, we can proceed with model training and evaluation.

__RU:__ _Теперь, когда у нас есть выбранные признаки в DataFrame df_selected, мы можем приступить к обучению и оценке модели._

In [21]:
feature_selection_task.predict(df_generated, result)[0].shape

(1200, 33)

# Make Pipeline
### Создание конвейера

In [22]:
try: 
    from tsad.base.pipeline import Pipeline
except:
    import sys
    sys.path.append('../')
    from tsad.base.pipeline import Pipeline

You have defined a combined pipeline using the `Pipeline` class, which allows you to define and execute multiple data processing tasks sequentially. The pipeline consists of two main tasks: feature generation and feature selection.

__RU:__ _Вы определили комбинированный конвейер с помощью класса Pipeline, который позволяет вам последовательно определять и выполнять несколько задач обработки данных. Конвейер состоит из двух основных задач: создания признаков и их выбора._

In [23]:
pipeline = Pipeline([
    FeatureGenerationTask(features=features, config=None),
    FeatureSelectionTask(target=target, 
                         remove_constant_features=True,
                         feature_selection_method='univariate', 
                         feature_selection_estimator='classifier'
                        ),
                    ]
)

__Feature Generation Task__

The first task in the pipeline is the `FeatureGenerationTask`. This task is responsible for generating new features from the input data. You can customize the features to generate by providing a list of feature names in the features parameter. In this case, config is set to None, indicating that the default configuration for feature generation will be used.

__Feature Selection Task__

The second task in the pipeline is the `FeatureSelectionTask`. This task is focused on selecting a subset of relevant features from the ones generated in the previous step.

__Задача создания признаков__

Первой задачей в конвейере является `FeatureGenerationTask`. Эта задача отвечает за создание новых признаков из входных данных. Вы можете настроить генерируемые признаки, указав список имен признаков в параметре Features. В этом случае для параметра config установлено значение None, что указывает на то, что для создания признаков будет использоваться конфигурация по умолчанию.

__Задача выбора признаков__

Вторая задача в конвейере — `FeatureSelectionTask`. Эта задача сосредоточена на выборе подмножества соответствующих признаков из созданных на предыдущем шаге.

## Fitting the Pipeline
#### Обучение конвейера

After defining the pipeline, you can fit it to your dataset using the `fit_predict` method:

__RU:__ _После определения конвейера вы можете обучить его на своем наборе данных с помощью метода `fit_predict`:_

In [24]:
%%time
df_fit = pipeline.fit_predict(df)
df_fit.shape

CPU times: user 2.39 s, sys: 518 ms, total: 2.91 s
Wall time: 17 s


(1200, 637)

Here, `df` represents input DataFrame. When you call `pipeline.fit(df)`, it performs the following steps:

1. The `FeatureGenerationTask` generates new features from the input DataFrame `df`.
2. The `FeatureSelectionTask` selects a subset of features based on the specified criteria, including removing constant features and using a classification-based estimator.
3. The resulting DataFrame with the selected features is stored in the variable `df_fit`.

__RU:__
Здесь `df` представляет собой входной DataFrame. Когда вы вызываете `pipeline.fit_predict(df)`, он выполняет следующие шаги:

1. FeatureGenerationTask генерирует новые признаки из входного DataFrame df.
2. FeatureSelectionTask выбирает подмножество признаков на основе заданных критериев, включая удаление постоянных признаков и использование средства оценки на основе классификации.
3. Полученный DataFrame с выбранными признаками сохраняется в переменной df_fit.

## Predicting with the Pipeline
#### Прогнозирование в конвейере

Once you have fitted your combined pipeline to your dataset using the `fit` method, you can also use it to make predictions. Here's how you can predict using the pipeline:

__RU:__ _После того как вы адаптировали комбинированный конвейер к набору данных с помощью метода `fit`, вы также можете использовать его для прогнозирования. Вот как можно прогнозировать с помощью конвейера:_

In [25]:
%%time
df_predict = pipeline.predict(df)
df_predict.shape

CPU times: user 323 ms, sys: 188 ms, total: 511 ms
Wall time: 3.47 s


(1200, 637)

The `predict` method is used to apply the trained pipeline to a new dataset (`df` in this case) and generate predictions or transformations based on the previously learned feature generation and selection steps.

__RU:__ _Метод `predict` используется для применения обученного конвейера к новому набору данных (в данном случае «df») и генерации прогнозов или преобразований на основе ранее изученных шагов генерации и выбора признаков._